In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
import numpy as np
import matplotlib.pyplot as plt

# ---- Load base model ----
weights_path = r"C:/Users/UMESH/OneDrive/Desktop/LP_4/Object Detection(Ass6)-20251108T124415Z-1-001/Object Detection(Ass6)/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
base = VGG16(weights=weights_path, include_top=False, input_shape=(224,224,3))

# ---- Freeze base model ----
for layer in base.layers:
    layer.trainable = False

# ---- Data generators ----
train_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
selected_classes = ['airplanes', 'ant']

train_data = train_gen.flow_from_directory(
   r"C:/Users/UMESH/OneDrive/Desktop/LP_4/Object Detection(Ass6)-20251108T124415Z-1-001/Object Detection(Ass6)/caltech-101-img",
    target_size=(224,224),
    batch_size=32,
    subset='training',
    classes=selected_classes
)

val_data = train_gen.flow_from_directory(
    r"C:/Users/UMESH/OneDrive/Desktop/LP_4/Object Detection(Ass6)-20251108T124415Z-1-001/Object Detection(Ass6)/caltech-101-img",
    target_size=(224,224),
    batch_size=32,
    subset='validation',
    classes=selected_classes
)

# ---- Model ----
model = Sequential([
    base,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(train_data.num_classes, activation='softmax')
])

model.compile(optimizer=Adam(), loss="categorical_crossentropy", metrics=["accuracy"])

# ---- Train ----
model.fit(train_data, validation_data=val_data, epochs=3)

# ---- Evaluate ----
loss, accuracy = model.evaluate(val_data)
print(f"Validation Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

# ---- Display predictions ----
images, labels = next(val_data)
predictions = model.predict(images)
pred_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(labels, axis=1)

plt.figure(figsize=(15,10))
for i in range(min(9, len(images))):
    plt.subplot(3,3,i+1)
    plt.imshow(images[i])
    plt.title(f"True: {selected_classes[true_classes[i]]}\nPred: {selected_classes[pred_classes[i]]}")
    plt.axis('off')
plt.tight_layout()
plt.show()

batch_acc = np.sum(pred_classes==true_classes)/len(images)*100
print(f"Batch Accuracy: {batch_acc:.2f}%")





Found 674 images belonging to 2 classes.
Found 168 images belonging to 2 classes.
Epoch 1/3


22/22 [==============================] - 304s 14s/step - loss: 0.0994 - accuracy: 0.9852 - val_loss: 0.0013 - val_accuracy: 1.0000
Epoch 3/3
4/6 [===================>..........] - ETA: 15s - loss: 7.6669e-06 - accuracy: 1.0000